In [33]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
import re


In [ ]:
df1 = pd.read_excel('data/CEFALOSPORINAS base de datos ADISINSIGHT/CEFACLOR.xlsx')

In [22]:
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns',None)

In [25]:
df1[df1['Number of Cases']==1].count()

Report Id                  22
Safety Report Name         22
Record Type                22
Release Date               22
Drugs                      22
Adverse Events             22
Mechanism of Action         0
Number of Cases            22
Serious                    22
First Report               22
Overdose                   22
Drug Interaction           22
Narrative and comments     22
Reporter Country           22
Source Country              0
Country of publication     22
Drug Class                 22
Age Group                  10
Route Of Administration     6
Treatment                   0
Special Situation           0
References                 22
URL                        22
dtype: int64

In [26]:
pd.set_option('display.max_colwidth', None)

In [27]:

df1[df1['Report Id']== 803578298][['Report Id','Narrative and comments']]


,Report Id,Narrative and comments
3,803578298,"In a single-center study of 476 patients, who underwent an allergy workup between January 1992 and July 2018, 86 patients [including 4 females and 1 male; aged 16−58 years; <EM>not all ages and sexes stated</EM>] were described, who developed drug hypersensitivity reaction (DHR) or DHR in the form of anaphylactic shock, anaphylaxis or urticaria during treatment with ceftriaxone, cefuroxime, cefotaxime, cefixime, cefpodoxime, cefepime, ceftazidime, cefatrizine, cefaclor, cefadroxil or cefazolin. Subsequently, five of the 86 patients developed DHR in the form of angioedema, bronchospasm, hypotension, collapse, loss of consciousness, exanthema, pruritus or malaise following administration of ampicillin, amoxicillin, benzylpenicillin, minor determinant mixture or penicilloyl polylysin for skin test (ST) or drug provocation test (DPT) [<EM>not all indications stated; dosages, routes, duration of treatments to reaction onsets and outcomes not stated</EM>].<BR />The patients were receiving cephalosporin treatment with ceftriaxone (16 patients), cefuroxime (14 patients), cefotaxime (5 patients), cefixime (2 patients), cefpodoxime (1 patient), cefepime (1 patient), ceftazidime (1 patient), cefatrizine (14 patients), cefaclor (5 patients), cefadroxil (1 patient) and cefazolin (26 patients). Subsequently, 81 of the 86 patients developed DHR secondary to the cephalosporins, and five patients developed DHR in the form of anaphylactic shock (2 patients), anaphylaxis (2 patients) and urticaria (1 patient) secondary to the cephalosporins. Hence, they underwent an allergy workup with ST or DPT. In 5 of the 86 patients, the ST were performed with cephalosporins at the dose of 2 mg/mL or penicillin reagents including penicilloyl polylysin, minor determinant mixture, amoxicillin or ampicillin at the dose of 20 mg/mL or benzylpenicillin 10000 UI/mL. Allergic work-up showed positive results for ceftriaxone, cefuroxime, cefotaxime, cefixime, cefpodoxime, cefepime, ceftazidime, cefatrizine, cefaclor, cefadroxil, cefazolin, minor determinant mixture, penicilloyl polylysin, benzylpenicillin [penicillin-G], amoxicillin or ampicillin, and negative result for minor determinant mixture, penicilloyl-polylysin, benzylpenicillin, amoxicillin, ampicillin, cefuroxime, cefepime, ceftazidime or cefotaxime. Based on ST or DPT, DHR was confirmed in all patients. Subsequently, five of the 86 patients developed hypersensitivity reaction in the form of angioedema, bronchospasm and hypotension (1 patient), hypotension, loss of consciousness and collapse (1 patient), exanthema (1 patient), exanthema, pruritus and malaise (1 patient), and pruritus (1 patient) secondary to cefradine, cefalotine, cefuroxime, cefotaxime, minor determinant mixture, penicilloyl polylysin, amoxicillin, ampicillin, benzylpenicillin, cefotaxime or ceftriaxone during ST.\n\nAuthor Comment\n&quot;Cephalosporin hypersensitivity was proven in 22.3% of the patients…by ST and DPT.&quot; &quot;Systemic reactions during skin testing occurred in 9.1% of positive patients&quot; &quot;[S]ystemic reactions in 5 cases&quot;"


In [28]:
df1[['Report Id','Narrative and comments']]

Report Id  \
0   803595494   
1   803589340   
2   803582224   
3   803578298   
4   803562921   
..        ...   
61  800505576   
62  800505547   
63  800504521   
64  800505368   
65  800504401   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       



Historial clínico unificado por paciente

✔ Excelente idea consolidar todo en un solo registro por paciente, con resumen de condiciones, tipo de fármaco y eventos adversos.


Por cuestiones de tiempo y por que es la entrada de datos que mejor manejará el modelo se hace un filtro donde el numeor de casos analizados sea 1, esto permite tener infromación puntual del paceinte. 

In [29]:
def solo_un_paciente(df):
    return  df[df['Number of Cases']==1]
    
#ejemplo el df 1 que utlizaremos para hacer la limpieza en este nb 

df = solo_un_paciente(df1)


Tipos de drogas recibidas

✔ No usar nombres específicos sino clases farmacológicas (antibióticos, antihistamínicos, corticoides, antifúngicos, antivirales, etc.).

✔ También con un diccionario tipo dict_drug_types para normalizar.


In [38]:
# para poder saber las posibles categorias en esto 
df_test = df[['Report Id','Drug Class']]
df_test

,Report Id,Drug Class
1,803589340,"Anti-infectives , Antibacterials , Antifungals , Antiprotozoals , Cephalosporins , Fluorobenzenes , Fluoroquinolones , Monobactams , Small molecules , Thiazoles , Triazoles"
5,803562657,"Amines , Analgesics , Antibacterials , Antineoplastics , Antipyretics , Antirheumatics , Antivirals , Cephalosporins , Hypoxanthines , Lincosamides , Nonsteroidal anti-inflammatories , Phenylpropionates , Small molecules"
11,803476613,"Antibacterials , Beta-lactams , Cephalosporins , Penicillins"
12,803475361,"Antibacterials , Cephalosporins"
14,803353779,"Antibacterials , Cephalosporins , Nonsteroidal anti-inflammatories adverse reactions (Serious)"
15,803347905,"Antibacterials , Cephalosporins , Penicillins , Small molecules"
18,803280356,"Antibacterials , Cephalosporins"
23,803165826,"Anti-ischaemics , Antibacterials , Anticoagulants , Cephalosporins , Coumarins , Small molecules"
24,803153340,"Antibacterials , Cephalosporins"
35,803058410,"Antibacterials , Cephalosporins"


In [ ]:

COL = 'Drug Class'  

# 1) Normalizar cada celda pero SIN romper conceptos internos
def normalize_phrase(p: str) -> str:
    s = p.upper()
    s = s.replace('_', ' ').replace('-', ' ')       # joiners -> espacio
    s = s.replace(' ADVERSE REACTIONS', '')         # ruido común
    s = s.replace(' (SERIOUS)', '') 
    s = re.sub(r'\s+', ' ', s).strip()              # espacios extra
    return s

def parse_classes(cell) -> list:
    if pd.isna(cell) or str(cell).strip()=='':
        return []
    # REGLA: solo separamos por coma (concepto por concepto)
    raw = [c.strip() for c in str(cell).split(',') if c.strip()]
    classes = [normalize_phrase(c) for c in raw]
    # deduplicar por registro y ordenar para estabilidad
    return sorted(set(classes))

# lista de clases por registro (frases)
df['Drug_Class_list'] = df[COL].apply(parse_classes)

# 2) Conteo de clases únicas (frases)
class_counts = (
    pd.Series([c for lst in df['Drug_Class_list'] for c in lst])
      .value_counts()
)
print(class_counts.head(30)) # quitar esto para poder solo tener el df sin print 



# 3) Dummies por clase (frase completa)
mlb = MultiLabelBinarizer()
drug_dummies = pd.DataFrame(
    mlb.fit_transform(df['Drug_Class_list']),
    columns=[c.replace(' ', '_') for c in mlb.classes_],  # columnas limpias
    index=df.index
)

df_test = pd.concat([df_test, drug_dummies], axis=1)


ANTIBACTERIALS                      22
CEPHALOSPORINS                      22
SMALL MOLECULES                      5
ANTI INFECTIVES                      5
NONSTEROIDAL ANTI INFLAMMATORIES     2
PENICILLINS                          2
ANTIPROTOZOALS                       1
ANTIFUNGALS                          1
MONOBACTAMS                          1
THIAZOLES                            1
TRIAZOLES                            1
FLUOROQUINOLONES                     1
AMINES                               1
ANALGESICS                           1
ANTIPYRETICS                         1
ANTINEOPLASTICS                      1
ANTIRHEUMATICS                       1
ANTIVIRALS                           1
HYPOXANTHINES                        1
FLUOROBENZENES                       1
LINCOSAMIDES                         1
PHENYLPROPIONATES                    1
BETA LACTAMS                         1
ANTI ISCHAEMICS                      1
ANTICOAGULANTS                       1
COUMARINS                

C:\Users\narro\AppData\Local\Temp\ipykernel_544040\895731679.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Drug_Class_list'] = df[COL].apply(parse_classes)


In [40]:
df_test

,Report Id,Drug Class,AMINES,AMINOGLYCOSIDES,ANALGESICS,ANTI_INFECTIVES,ANTI_INFLAMMATORIES,ANTI_ISCHAEMICS,ANTIBACTERIALS,ANTICOAGULANTS,ANTIFUNGALS,ANTIMALARIALS,ANTINEOPLASTICS,ANTIPROTOZOALS,ANTIPYRETICS,ANTIRHEUMATICS,ANTIVIRALS,BETA_LACTAMS,CEPHALOSPORINS,COUMARINS,DISEASE_MODIFYING_ANTIRHEUMATICS,FLUOROBENZENES,FLUOROQUINOLONES,HYPOXANTHINES,LINCOSAMIDES,MONOBACTAMS,NONSTEROIDAL_ANTI_INFLAMMATORIES,PENICILLINS,PEPTIDE_ANTIBIOTICS,PEPTIDES,PHENYLPROPIONATES,SALICYLATES,SKIN_DISORDER_THERAPIES,SMALL_MOLECULES,SULFONAMIDES,TETRACYCLINES,THIAZOLES,TRIAZOLES
1,803589340,"Anti-infectives , Antibacterials , Antifungals , Antiprotozoals , Cephalosporins , Fluorobenzenes , Fluoroquinolones , Monobactams , Small molecules , Thiazoles , Triazoles",0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1
5,803562657,"Amines , Analgesics , Antibacterials , Antineoplastics , Antipyretics , Antirheumatics , Antivirals , Cephalosporins , Hypoxanthines , Lincosamides , Nonsteroidal anti-inflammatories , Phenylpropionates , Small molecules",1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0
11,803476613,"Antibacterials , Beta-lactams , Cephalosporins , Penicillins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
12,803475361,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14,803353779,"Antibacterials , Cephalosporins , Nonsteroidal anti-inflammatories adverse reactions (Serious)",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
15,803347905,"Antibacterials , Cephalosporins , Penicillins , Small molecules",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
18,803280356,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
23,803165826,"Anti-ischaemics , Antibacterials , Anticoagulants , Cephalosporins , Coumarins , Small molecules",0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
24,803153340,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,803058410,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Interacciones entre drogas

Si algún paciente tiene múltiples tipos de fármacos en el mismo episodio, crea una variable polytherapy = 1.

In [42]:
drug_dummies.columns.to_list()

['AMINES',
 'AMINOGLYCOSIDES',
 'ANALGESICS',
 'ANTI_INFECTIVES',
 'ANTI_INFLAMMATORIES',
 'ANTI_ISCHAEMICS',
 'ANTIBACTERIALS',
 'ANTICOAGULANTS',
 'ANTIFUNGALS',
 'ANTIMALARIALS',
 'ANTINEOPLASTICS',
 'ANTIPROTOZOALS',
 'ANTIPYRETICS',
 'ANTIRHEUMATICS',
 'ANTIVIRALS',
 'BETA_LACTAMS',
 'CEPHALOSPORINS',
 'COUMARINS',
 'DISEASE_MODIFYING_ANTIRHEUMATICS',
 'FLUOROBENZENES',
 'FLUOROQUINOLONES',
 'HYPOXANTHINES',
 'LINCOSAMIDES',
 'MONOBACTAMS',
 'NONSTEROIDAL_ANTI_INFLAMMATORIES',
 'PENICILLINS',
 'PEPTIDE_ANTIBIOTICS',
 'PEPTIDES',
 'PHENYLPROPIONATES',
 'SALICYLATES',
 'SKIN_DISORDER_THERAPIES',
 'SMALL_MOLECULES',
 'SULFONAMIDES',
 'TETRACYCLINES',
 'THIAZOLES',
 'TRIAZOLES']

In [44]:
# si guardaste la lista antes: drug_cols = list(drug_dummies.columns)
drug_cols = list(drug_dummies.columns)

df_test['n_drug_classes'] = df_test[drug_cols].sum(axis=1)
df_test['polytherapy']    = (df_test['n_drug_classes'] > 1).astype(int)
df_test


,Report Id,Drug Class,AMINES,AMINOGLYCOSIDES,ANALGESICS,ANTI_INFECTIVES,ANTI_INFLAMMATORIES,ANTI_ISCHAEMICS,ANTIBACTERIALS,ANTICOAGULANTS,ANTIFUNGALS,ANTIMALARIALS,ANTINEOPLASTICS,ANTIPROTOZOALS,ANTIPYRETICS,ANTIRHEUMATICS,ANTIVIRALS,BETA_LACTAMS,CEPHALOSPORINS,COUMARINS,DISEASE_MODIFYING_ANTIRHEUMATICS,FLUOROBENZENES,FLUOROQUINOLONES,HYPOXANTHINES,LINCOSAMIDES,MONOBACTAMS,NONSTEROIDAL_ANTI_INFLAMMATORIES,PENICILLINS,PEPTIDE_ANTIBIOTICS,PEPTIDES,PHENYLPROPIONATES,SALICYLATES,SKIN_DISORDER_THERAPIES,SMALL_MOLECULES,SULFONAMIDES,TETRACYCLINES,THIAZOLES,TRIAZOLES,n_drug_classes,polytherapy
1,803589340,"Anti-infectives , Antibacterials , Antifungals , Antiprotozoals , Cephalosporins , Fluorobenzenes , Fluoroquinolones , Monobactams , Small molecules , Thiazoles , Triazoles",0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,11,1
5,803562657,"Amines , Analgesics , Antibacterials , Antineoplastics , Antipyretics , Antirheumatics , Antivirals , Cephalosporins , Hypoxanthines , Lincosamides , Nonsteroidal anti-inflammatories , Phenylpropionates , Small molecules",1,0,1,0,0,0,1,0,0,0,1,0,1,1,1,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,1,0,0,0,0,13,1
11,803476613,"Antibacterials , Beta-lactams , Cephalosporins , Penicillins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,4,1
12,803475361,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
14,803353779,"Antibacterials , Cephalosporins , Nonsteroidal anti-inflammatories adverse reactions (Serious)",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,3,1
15,803347905,"Antibacterials , Cephalosporins , Penicillins , Small molecules",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,4,1
18,803280356,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
23,803165826,"Anti-ischaemics , Antibacterials , Anticoagulants , Cephalosporins , Coumarins , Small molecules",0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,6,1
24,803153340,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1
35,803058410,"Antibacterials , Cephalosporins",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1




Comorbilidades

✔ Crear variables dummy (0/1) por presencia de comorbilidad agrupada (cardíaca, hepática, respiratoria, neurológica, inmunológica, etc.).

✔ Basarte en una clasificación tipo MedDRA / WHO-ART / ICD-10 simplificada es correcto.

✔ Usa un diccionario de mapeo (dict_comorbilidades) que contenga términos y sinónimos → categoría madre.
Ejemplo:

dict_comorbilidades = {
    'diabetes': 'endocrine_disorder',
    'hipertensión': 'cardiac_disorder',
    'epilepsia': 'nervous_system_disorder'
}



In [45]:
df_nar = df[['Report Id','Narrative and comments']]
df_nar

,Report Id,Narrative and comments
1,803589340,"A 66-year-old man developed erythro-papular-type drug eruption during treatment with cefaclor, levofloxacin and fosravuconazole for onychomycosis, and drug-induced skin eruption during treatment with aztreonam [<EM>not all indications, dosages and routes stated</EM>].<BR />The man, who had cataract, was admitted and underwent cataract surgery in July 2019 in Japan. He received cefaclor on the day of surgery, and oedematous erythema appeared on his body the following day. A drug eruption due to cefaclor was suspected and he was switched to levofloxacin.Three days after the first examination, he was re-examined and the erythema on whole body was observed. He had oedematous erythema with no itchiness, which was mainly presented on his torso and lower limbs. On his trunks, a palpable,scattered erythema was observed. He had no fever. Laboratory tests revealed the following: WBC 8300 /µL (eosinophils 4%, neutrophils 83%, lymphocytes 8% and monocytes 5%). Histopathology showed liquefaction. However, in the skin lesion necrotic keratinocytes were not detected.<BR />The man received treatment with prednisolone for drug-induced eruption. Initially, his symptoms improved. However, the oedematous erythema eventually resumed spreading. Previously, he had developed drug-induced skin eruption with aztreonam, for which he was hospitalised and recovered. His medical history was significant for onychomycosis, for which he had been receiving oral fosravuconazole [fosravuconazole L-lysine ethanolate] since 41 days. He was also on other concomitant medications. His fosravuconazole treatment was then stopped. The erythema resolved and he continued treatment with prednisolone. After one month of treatment, prednisolone was discontinued. The onychomycosis was treated with efinaconazole solution after discontinuation of fosravuconazole. Lymphocyte transformation test (LTT) was conducted for cefaclor and levofloxacin on day 6 after the drug eruption occurred, and for fosravuconazole on day 20. Stimulation indices for cefaclor, levofloxacin, and fosravuconazole were 97%, 108%, and 189%, respectively. Based on the clinical presentation and various findings, he was diagnosed with cefaclor, levofloxacin and fosravuconazole-induced erythro-papular-type drug eruption. No further recurrence of erythema was observed.\n\nAuthor Comment\n&quot;We report a case of erythro-papular drug eruption caused by [fosravuconazole].&quot; &quot;He had experienced a drug-induced skin eruption with aztreonam.&quot; &quot;He started systemic treatment with cefaclor…oedematous erythema appeared on his body the following day. Cefaclor…caused drug eruptions…and he was switched to levofloxacin.&quot; &quot;He was treated with prednisolone at 30 mg/day for the prolonged drug induced skin eruption.&quot;"
5,803562657,"A 14-year-old boy exhibited lack of efficacy with cefaclor, clindamycin, ibuprofen and ganciclovir, while being treated for multilobar pneumonia [<EM>dosages and routes not stated</EM>].<BR />The boy was admitted with symptoms of cough, hemoptysis, nausea and chest pain. Subsequently, he started receiving treatment with cefaclor. Four days during the treatment, his general condition deteriorated and it was observed that he had a fever with associated chills. Thereafter, he started receiving treatment with clindamycin, ibuprofen [Brufen] and ganciclovir in addition to cefaclor. After a day, he had trouble breathing, which worse when lying flat. However, despite these treatments his general condition deteriorated contineously. A chest CT scan revaled multilobar pneumonia, which was more severe on the left upper lobe. Due to his worsening multilobar pneumonia despite treatment with cefaclor, clindamycin, ibuprofen and ganciclovir (lack of efficacy) he was shifted to EICU for further management. After further investigations, he was diagnosed with severe pneumonia, sepsis, moderate pulmonary hypertension (PH) and left ventricul

Número de comorbilidades

Una variable continua o categorizada (0, 1–2, ≥3) puede servir como control de complejidad clínica.


Edad y género + categorías etarias

✔ Perfecto generar una variable categórica derivada (infante, niño, adolescente, adulto, adulto mayor, embarazada).

✔ Puedes usar algo como:

def clasificar_edad(edad, genero, embarazada):
    if embarazada: return "embarazada"
    elif edad < 1: return "lactante"
    elif edad < 12: return "niño"
    elif edad < 18: return "adolescente"
    elif edad < 60: return "adulto"
    else: return "adulto_mayor"


In [47]:
df_nar['Narrative_clean'] = df_nar['Narrative and comments'].str.lower()


def extract_age(text):
    if not isinstance(text, str):
        return None
    # Buscar "X-year-old"
    year_match = re.search(r'(\d+)\s*[-]?\s*year[- ]?old', text)
    if year_match:
        return int(year_match.group(1))
    # Buscar "X-month-old"
    month_match = re.search(r'(\d+)\s*[-]?\s*month[- ]?old', text)
    if month_match:
        return round(int(month_match.group(1)) / 12, 2)
    return None

df_nar['Age'] = df_nar['Narrative_clean'].apply(extract_age)


C:\Users\narro\AppData\Local\Temp\ipykernel_544040\525756804.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Narrative_clean'] = df_nar['Narrative and comments'].str.lower()
C:\Users\narro\AppData\Local\Temp\ipykernel_544040\525756804.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Age'] = df_nar['Narrative_clean'].apply(extract_age)


In [49]:
female_keywords = ['female', 'woman', 'girl', 'lady']
male_keywords = ['male', 'man', 'boy', 'gentleman']

def extract_gender(text):
    if not isinstance(text, str):
        return 'non-specified'
    for word in female_keywords:
        if word in text:
            return 'female'
    for word in male_keywords:
        if word in text:
            return 'male'
    return 'non-specified'

df_nar['Gender'] = df_nar['Narrative_clean'].apply(extract_gender)


C:\Users\narro\AppData\Local\Temp\ipykernel_544040\3934763963.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Gender'] = df_nar['Narrative_clean'].apply(extract_gender)




Temperatura corporal

✔ Incluirla como variable numérica o categórica (normal / febril / hipertermia).

✔ Puedes derivar una variable binaria fiebre = 1 if temp >= 38 else 0.


In [51]:
import re
import pandas as pd
import numpy as np

# --- 1) Extracción de temperatura numérica ---
# Captura formas comunes: '39°C', '39 °c', 'temp 39.2 c', 'temperature 102 f', 't=38.5°c'
cel_pat = re.compile(r'(?:(?:temp(?:erature)?|t)[^\d]{0,5})?(\d{1,2}(?:[.,]\d{1,2})?)\s*°?\s*(?:c|celsius)\b')
fah_pat = re.compile(r'(?:(?:temp(?:erature)?|t)[^\d]{0,5})?(\d{2,3}(?:[.,]\d{1,2})?)\s*°?\s*(?:f|fahrenheit)\b')
# Casos sin unidad pero con "°" y rango razonable (25–45)
bare_deg_pat = re.compile(r'(\d{1,2}(?:[.,]\d{1,2})?)\s*°\b')

def _to_float(num_str):
    try:
        return float(num_str.replace(',', '.'))
    except Exception:
        return None

def extract_temp_c(text: str):
    if not isinstance(text, str) or not text:
        return np.nan

    # 1) Celsius explícito
    m = cel_pat.search(text)
    if m:
        val = _to_float(m.group(1))
        # Filtro de plausibilidad humana
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    # 2) Fahrenheit explícito -> convertir
    m = fah_pat.search(text)
    if m:
        f = _to_float(m.group(1))
        if f is not None and 80 <= f <= 113:  # ~26.7–45°C
            c = (f - 32) * 5/9
            return round(c, 2)

    # 3) Número con símbolo ° sin unidad, validar rango humano
    m = bare_deg_pat.search(text)
    if m:
        val = _to_float(m.group(1))
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    # 4) Patrones tipo "38-39°c" -> tomar el primero
    range_cel = re.search(r'(\d{1,2}(?:[.,]\d{1,2})?)\s*[-–]\s*\d{1,2}(?:[.,]\d{1,2})?\s*°?\s*(?:c|celsius)\b', text)
    if range_cel:
        val = _to_float(range_cel.group(1))
        if val is not None and 25 <= val <= 45:
            return round(val, 2)

    return np.nan

df_nar['Temp_C'] = df_nar['Narrative_clean'].apply(extract_temp_c)

# --- 2) Clasificación clínica ---
# Umbrales (ajústalos si quieres):
# <35.0 -> hypothermia ; 35.0–37.4 -> normal ; >=37.5 -> fever
def classify_temp_from_value(c):
    if pd.isna(c):
        return None
    if c < 35.0:
        return 'hypothermia'
    if c >= 37.5:
        return 'fever'
    return 'normal'

# Palabras clave cuando no hay número
# Nota: primero checamos negaciones como "afebrile", "no fever"
normal_kw = [
    'afebrile', 'apyrexial', 'apyrexic', 'no fever', 'without fever', 'defervesc',
]
fever_kw = [
    'fever', 'febrile', 'pyrexia', 'high temperature', 'high-grade fever', 'low-grade fever'
]
hypo_kw = ['hypothermia', 'hypothermic']

def classify_temp_from_text(text: str):
    if not isinstance(text, str) or not text:
        return 'non-specified'
    # normal (negaciones primero)
    for w in normal_kw:
        if w in text:
            return 'normal'
    # hypothermia
    for w in hypo_kw:
        if w in text:
            return 'hypothermia'
    # fever
    for w in fever_kw:
        if w in text:
            return 'fever'
    return 'non-specified'

def final_temp_category(row):
    c = row['Temp_C']
    if pd.notna(c):
        return classify_temp_from_value(c)
    # sin número: inferir por texto
    return classify_temp_from_text(row['Narrative_clean'])

df_nar['Temp_Category'] = df_nar.apply(final_temp_category, axis=1)

df_nar


C:\Users\narro\AppData\Local\Temp\ipykernel_544040\863512229.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Temp_C'] = df_nar['Narrative_clean'].apply(extract_temp_c)
C:\Users\narro\AppData\Local\Temp\ipykernel_544040\863512229.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Temp_Category'] = df_nar.apply(final_temp_category, axis=1)


,Report Id,Narrative and comments,Narrative_clean,Age,Gender,Temp_C,Temp_Category
1,803589340,"A 66-year-old man developed erythro-papular-type drug eruption during treatment with cefaclor, levofloxacin and fosravuconazole for onychomycosis, and drug-induced skin eruption during treatment with aztreonam [<EM>not all indications, dosages and routes stated</EM>].<BR />The man, who had cataract, was admitted and underwent cataract surgery in July 2019 in Japan. He received cefaclor on the day of surgery, and oedematous erythema appeared on his body the following day. A drug eruption due to cefaclor was suspected and he was switched to levofloxacin.Three days after the first examination, he was re-examined and the erythema on whole body was observed. He had oedematous erythema with no itchiness, which was mainly presented on his torso and lower limbs. On his trunks, a palpable,scattered erythema was observed. He had no fever. Laboratory tests revealed the following: WBC 8300 /µL (eosinophils 4%, neutrophils 83%, lymphocytes 8% and monocytes 5%). Histopathology showed liquefaction. However, in the skin lesion necrotic keratinocytes were not detected.<BR />The man received treatment with prednisolone for drug-induced eruption. Initially, his symptoms improved. However, the oedematous erythema eventually resumed spreading. Previously, he had developed drug-induced skin eruption with aztreonam, for which he was hospitalised and recovered. His medical history was significant for onychomycosis, for which he had been receiving oral fosravuconazole [fosravuconazole L-lysine ethanolate] since 41 days. He was also on other concomitant medications. His fosravuconazole treatment was then stopped. The erythema resolved and he continued treatment with prednisolone. After one month of treatment, prednisolone was discontinued. The onychomycosis was treated with efinaconazole solution after discontinuation of fosravuconazole. Lymphocyte transformation test (LTT) was conducted for cefaclor and levofloxacin on day 6 after the drug eruption occurred, and for fosravuconazole on day 20. Stimulation indices for cefaclor, levofloxacin, and fosravuconazole were 97%, 108%, and 189%, respectively. Based on the clinical presentation and various findings, he was diagnosed with cefaclor, levofloxacin and fosravuconazole-induced erythro-papular-type drug eruption. No further recurrence of erythema was observed.\n\nAuthor Comment\n&quot;We report a case of erythro-papular drug eruption caused by [fosravuconazole].&quot; &quot;He had experienced a drug-induced skin eruption with aztreonam.&quot; &quot;He started systemic treatment with cefaclor…oedematous erythema appeared on his body the following day. Cefaclor…caused drug eruptions…and he was switched to levofloxacin.&quot; &quot;He was treated with prednisolone at 30 mg/day for the prolonged drug induced skin eruption.&quot;","a 66-year-old man developed erythro-papular-type drug eruption during treatment with cefaclor, levofloxacin and fosravuconazole for onychomycosis, and drug-induced skin eruption during treatment with aztreonam [<em>not all indications, dosages and routes stated</em>].<br />the man, who had cataract, was admitted and underwent cataract surgery in july 2019 in japan. he received cefaclor on the day of surgery, and oedematous erythema appeared on his body the following day. a drug eruption due to cefaclor was suspected and he was switched to levofloxacin.three days after the first examination, he was re-examined and the erythema on whole body was observed. he had oedematous erythema with no itchiness, which was mainly presented on his torso and lower limbs. on his trunks, a palpable,scattered erythema was observed. he had no fever. laboratory tests revealed the following: wbc 8300 /µl (eosinophils 4%, neutrophils 83%, lymphocytes 8% and monocytes 5%). histopathology showed liquefaction. however, in the skin lesion necrotic keratinocytes were not detected.<br />the man received treatment wit


Tipo de infección

✔ Excelente incluir el agente causal (bacteria, virus, hongo, parásito).

✔ Conviene hacer un diccionario dict_infecciones con palabras clave (por ejemplo: "staph", "mycobacterium" → bacterial; "candida" → fungal).


In [52]:

# --- Listas de palabras clave por categoría ---
kw = {
    "fungus": [
        r"\bfungal\b", r"\bmycosis\b", r"\bonychomycosis\b", r"\botomycosis\b",
        r"\bcandid(?:a|iasis)\b", r"\baspergillus\b", r"\bcryptococcus\b",
        r"\bhistoplasma\b", r"\bblastomyces\b", r"\bcoccidioides\b",
        r"\bdermatophyte\b", r"\btinea\b", r"\bmucor(?:mycosis)?\b",
        r"\bsporothrix\b"
    ],
    "parasite": [
        r"\bparasite|parasitic|parasitism\b", r"\bprotozoa(?:n|l)?\b", r"\bhelminth\b",
        r"\bmalaria\b", r"\bplasmodium\b", r"\bgiardi(?:a|asis)\b",
        r"\bamoeb(?:a|iasis)\b", r"\bentamoeba\b", r"\btrichomonas\b",
        r"\btoxoplasma(?:osis)?\b", r"\bleishmania(?:sis)?\b",
        r"\btaeni(?:a|asis)\b", r"\bechinococcus\b", r"\bschistosom(?:a|iasis)\b",
        r"\bstrongyloides\b", r"\bscabies\b", r"\bpediculosis\b", r"\bpinworm\b", r"\bhookworm\b"
    ],
    "virus": [
        r"\bviral\b", r"\bvir(?:us|al) infection\b", r"\binfluenza\b", r"\bflu\b",
        r"\bvaricella\b", r"\bherpes(?:\s(simplex|zoster))?\b", r"\brsv\b",
        r"\bsars[- ]?cov[- ]?2\b", r"\bcovid[- ]?19\b", r"\bhiv\b",
        r"\bhbv\b", r"\bhcv\b", r"\bhpv\b", r"\bebv\b", r"\bcmv\b",
        r"\bnorovirus\b", r"\badenovirus\b", r"\brotavirus\b", r"\benterovirus\b",
        r"\bc(orona)?virus\b"
    ],
    "bacteria": [
        r"\bbacterial\b", r"\bbacteremia\b", r"\bcellulitis\b", r"\bpyelonephritis\b",
        r"\bmrsa\b", r"\bmssa\b", r"\bstaph(?:ylococcus)?\b", r"\bstrept(?:ococcus|ococcal|ococcus)\b",
        r"\bpseudomonas\b", r"\bescherichia\s*coli\b|\becoli\b", r"\benterobacter\b",
        r"\bklebs(?:iella)?\b", r"\bproteus\b", r"\bacinetobacter\b", r"\bserratia\b",
        r"\bclostrid(?:ium|ioides)\b", r"\blisteria|listeria\b", r"\bmycobacter(?:ium|ia)\b",
        r"\bgram[- ]?(positive|negative)\b", r"\buti\b", r"\burninary tract infection\b"
    ],
}

# Compilar patrones por categoría para rendimiento
compiled = {cat: re.compile("|".join(pats)) for cat, pats in kw.items()}

def detect_infection_types(text: str):
    if not isinstance(text, str) or not text:
        return set()
    found = set()
    for cat, pat in compiled.items():
        if pat.search(text):
            found.add(cat)
    return found

def collapse_to_category(found: set):
    if not found:
        return "non-specified"
    if len(found) == 1:
        return list(found)[0]
    return "mixed"

# Columna categórica principal
df_nar['Infection_Type'] = df_nar['Narrative_clean'].apply(lambda t: collapse_to_category(detect_infection_types(t)))

# (Opcional) one-hot por categoría, por si te sirve para análisis
for cat in kw.keys():
    df_nar[f'is_{cat}'] = df_nar['Narrative_clean'].apply(lambda t: 1 if compiled[cat].search(t or "") else 0)

# Inspección rápida:
# df['Infection_Type'].value_counts()
# df[['Infection_Type', 'is_fungus','is_parasite','is_virus','is_bacteria']].head()


C:\Users\narro\AppData\Local\Temp\ipykernel_544040\1441721466.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar['Infection_Type'] = df_nar['Narrative_clean'].apply(lambda t: collapse_to_category(detect_infection_types(t)))
C:\Users\narro\AppData\Local\Temp\ipykernel_544040\1441721466.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar[f'is_{cat}'] = df_nar['Narrative_clean'].apply(lambda t: 1 if compiled[cat].search(t or "") else 0)
C:\Users\narro\AppData\Local\Temp\ipykernel_544040\144172

In [53]:
df_nar

,Report Id,Narrative and comments,Narrative_clean,Age,Gender,Temp_C,Temp_Category,Infection_Type,is_fungus,is_parasite,is_virus,is_bacteria
1,803589340,"A 66-year-old man developed erythro-papular-type drug eruption during treatment with cefaclor, levofloxacin and fosravuconazole for onychomycosis, and drug-induced skin eruption during treatment with aztreonam [<EM>not all indications, dosages and routes stated</EM>].<BR />The man, who had cataract, was admitted and underwent cataract surgery in July 2019 in Japan. He received cefaclor on the day of surgery, and oedematous erythema appeared on his body the following day. A drug eruption due to cefaclor was suspected and he was switched to levofloxacin.Three days after the first examination, he was re-examined and the erythema on whole body was observed. He had oedematous erythema with no itchiness, which was mainly presented on his torso and lower limbs. On his trunks, a palpable,scattered erythema was observed. He had no fever. Laboratory tests revealed the following: WBC 8300 /µL (eosinophils 4%, neutrophils 83%, lymphocytes 8% and monocytes 5%). Histopathology showed liquefaction. However, in the skin lesion necrotic keratinocytes were not detected.<BR />The man received treatment with prednisolone for drug-induced eruption. Initially, his symptoms improved. However, the oedematous erythema eventually resumed spreading. Previously, he had developed drug-induced skin eruption with aztreonam, for which he was hospitalised and recovered. His medical history was significant for onychomycosis, for which he had been receiving oral fosravuconazole [fosravuconazole L-lysine ethanolate] since 41 days. He was also on other concomitant medications. His fosravuconazole treatment was then stopped. The erythema resolved and he continued treatment with prednisolone. After one month of treatment, prednisolone was discontinued. The onychomycosis was treated with efinaconazole solution after discontinuation of fosravuconazole. Lymphocyte transformation test (LTT) was conducted for cefaclor and levofloxacin on day 6 after the drug eruption occurred, and for fosravuconazole on day 20. Stimulation indices for cefaclor, levofloxacin, and fosravuconazole were 97%, 108%, and 189%, respectively. Based on the clinical presentation and various findings, he was diagnosed with cefaclor, levofloxacin and fosravuconazole-induced erythro-papular-type drug eruption. No further recurrence of erythema was observed.\n\nAuthor Comment\n&quot;We report a case of erythro-papular drug eruption caused by [fosravuconazole].&quot; &quot;He had experienced a drug-induced skin eruption with aztreonam.&quot; &quot;He started systemic treatment with cefaclor…oedematous erythema appeared on his body the following day. Cefaclor…caused drug eruptions…and he was switched to levofloxacin.&quot; &quot;He was treated with prednisolone at 30 mg/day for the prolonged drug induced skin eruption.&quot;","a 66-year-old man developed erythro-papular-type drug eruption during treatment with cefaclor, levofloxacin and fosravuconazole for onychomycosis, and drug-induced skin eruption during treatment with aztreonam [<em>not all indications, dosages and routes stated</em>].<br />the man, who had cataract, was admitted and underwent cataract surgery in july 2019 in japan. he received cefaclor on the day of surgery, and oedematous erythema appeared on his body the following day. a drug eruption due to cefaclor was suspected and he was switched to levofloxacin.three days after the first examination, he was re-examined and the erythema on whole body was observed. he had oedematous erythema with no itchiness, which was mainly presented on his torso and lower limbs. on his trunks, a palpable,scattered erythema was observed. he had no fever. laboratory tests revealed the following: wbc 8300 /µl (eosinophils 4%, neutrophils 83%, lymphocytes 8% and monocytes 5%). histopathology showed liquefaction. however, in the skin lesion necrotic keratinocy

Ruta de administración

✔ Una variable tipo (oral, IV, tópica, etc.) puede ser relevante para reacciones neurológicas.


In [55]:
df.columns

Index(['Report Id', 'Safety Report Name', 'Record Type', 'Release Date',
       'Drugs', 'Adverse Events', 'Mechanism of Action', 'Number of Cases',
       'Serious', 'First Report', 'Overdose', 'Drug Interaction',
       'Narrative and comments', 'Reporter Country', 'Source Country',
       'Country of publication', 'Drug Class', 'Age Group',
       'Route Of Administration', 'Treatment', 'Special Situation',
       'References', 'URL', 'Drug_Class_list'],
      dtype='object')

In [ ]:
import re
import pandas as pd
import numpy as np

# --- 1) Lista de cefalosporinas (incluye variantes ES/EN) ---
cephalosporins = [
    r"cefaclor",
    r"cefalexina|cephalexin",
    r"cefazolina|cefazolin",
    r"cefepime",
    r"cefixima|cefixime",
    r"cefoperazona|cefoperazone",
    r"cefotaxima|cefotaxime",
    r"cefotetan",
    r"cefoxitina|cefoxitin",
    r"ceftarolina|ceftaroline",
    r"ceftazidima(?:\s+con\s+avibactam)?|ceftazidime(?:\s+with\s+avibactam)?|ceftazidime\s*/\s*avibactam",
    r"ceftibuten",
    r"ceftolozano|ceftolozane",
    r"ceftriaxona|ceftriaxone",
    r"cefuroxima|cefuroxime",
]
drug_pat = re.compile(r"(" + r"|".join(cephalosporins) + r")\b", flags=re.I)

# --- 2) Patrones de rutas de administración ---
route_map = {
    "oral": [
        r"\boral\b", r"\bpo\b", r"\bper\s*os\b", r"\bby\s*mouth\b", r"\bvo\b"
    ],
    "iv": [
        r"\bintraven(?:ous|a)\b", r"\biv\b", r"\biv\s*(bolus|infusion|drip)\b",
        r"\bpre[- ]?op(?:erative)?\s+prophylaxis\b"  # suele ser IV en quirófano
    ],
    "im": [r"\bintramuscular\b", r"\bim\b"],
    "sc": [r"\bsubcut(?:aneous|a)\b", r"\bsc\b", r"\bsq\b"],
    "topical": [r"\btopical\b", r"\bointment\b", r"\bcream\b", r"\bgel\b"],
    "ophthalmic": [r"\bophthalmic\b", r"\beye\s*drops?\b"],
    "otic": [r"\botic\b", r"\bear\s*drops?\b"],
    "intrathecal": [r"\bintrathecal\b", r"\bit\b"],
    "inhaled": [r"\binhal(?:ed|ation)\b", r"\bnebuliz(?:ed|er|ación)\b"],
    "rectal": [r"\brectal\b", r"\bpr\b", r"\bsuppositor(y|ies)\b"],
    "vaginal": [r"\bvaginal\b", r"\bpv\b"],
    "transdermal": [r"\btransdermal\b", r"\bpatch\b"],
    "intraperitoneal": [r"\bintraperitoneal\b", r"\bip\b"],
    "intraarticular": [r"\bintra[- ]?articular\b", r"\bia\b"],
}
compiled_routes = {k: re.compile("|".join(v)) for k, v in route_map.items()}

def find_routes(text: str):
    """Devuelve set de rutas encontradas y origen ('near-drug' o 'global')."""
    if not isinstance(text, str) or not text:
        return set(), None

    # 1) buscar menciones de drogas y revisar ventana cercana
    routes = set()
    for m in drug_pat.finditer(text):
        start, end = m.span()
        window = text[max(0, start-140): min(len(text), end+140)]
        for route, pat in compiled_routes.items():
            if pat.search(window):
                routes.add(route)

    if routes:
        return routes, "near-drug"

    # 2) búsqueda global si no se halló cerca del fármaco
    for route, pat in compiled_routes.items():
        if pat.search(text):
            routes.add(route)
    if routes:
        return routes, "global"

    return set(), None

# Para textos que dicen "was given" sin ruta pero mencionan "tablet/capsule/iv line"
infer_map = {
    "oral": re.compile(r"\btablet|capsule|suspension\b"),
    "iv": re.compile(r"\bline|cannula|infus(?:ion|ed)\b"),
}
def infer_route(text: str):
    if not isinstance(text, str) or not text:
        return None
    for r, pat in infer_map.items():
        if pat.search(text):
            return r
    return None

def collapse_routes(routes: set):
    if not routes:
        return "non-specified"
    if len(routes) == 1:
        return list(routes)[0]
    return "multiple"

def extract_admin_route(text: str):
    routes, origin = find_routes(text)
    confidence = None
    if routes:
        confidence = "high" if origin == "near-drug" else "medium"
    else:
        # intento de inferencia débil
        guessed = infer_route(text)
        if guessed:
            routes = {guessed}
            confidence = "low"
    return collapse_routes(routes), origin or ("inferred" if routes else None), confidence

# --- columnas derivadas ---
tmp = df_nar['Narrative_clean'].apply(extract_admin_route)
df_nar[['Admin_Route', 'Admin_Route_source', 'Admin_Route_confidence']] = pd.DataFrame(tmp.tolist(), index=df.index)

# (Opcional) indicadores binarios por ruta para análisis
#for r in route_map.keys():
 #   df_nar[f'route_{r}'] = df_nar['Narrative_clean'].apply(lambda t: 1 if compiled_routes[r].search(t or "") else 0)

# (Opcional) flag: menciona alguna cefalosporina
df_nar['mentions_cephalosporin'] = df_nar['Narrative_clean'].str.contains(drug_pat).fillna(False)


C:\Users\narro\AppData\Local\Temp\ipykernel_544040\1295844767.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar[['Admin_Route', 'Admin_Route_source', 'Admin_Route_confidence']] = pd.DataFrame(tmp.tolist(), index=df.index)
C:\Users\narro\AppData\Local\Temp\ipykernel_544040\1295844767.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nar[['Admin_Route', 'Admin_Route_source', 'Admin_Route_confidence']] = pd.DataFrame(tmp.tolist(), index=df.index)
C:\Users\narro\AppData\Local\Temp\ipykernel_54

In [58]:
df[['Report Id', 'Route Of Administration']]

,Report Id,Route Of Administration
1,803589340,Oral
5,803562657,NaN
11,803476613,"Intravenous, Oral"
12,803475361,Oral
14,803353779,NaN
15,803347905,Oral
18,803280356,Oral
23,803165826,Oral
24,803153340,NaN
35,803058410,NaN



Variable objetivo: convulsiones

✔ Muy buena definición de target binario (1 si hubo convulsión, 0 si no).

✔ Esto permitirá modelar riesgo de convulsión por tipo de fármaco o condición.



In [63]:

# --- Palabras clave de convulsiones ---
seizure_kw = [
    r"\bseizure(s)?\b",
    r"\bconvulsion(s)?\b",
    r"\bepileptic\s+(fit|seizure|attack)\b",
    r"\bepilepsy\b",
    r"\bstatus\s+epilepticus\b",
    r"\btonic[- ]?clonic\b",
    r"\bgrand\s+mal\b",
    r"\bfebrile\s+seizure\b",
    r"\babsence\s+seizure\b",
    r"\bmyoclonic\b",
    r"\bclonic\b",
    r"\btonic\b",
    r"\bataque\s+epiléptico\b",
    r"\bcrisis\s+epiléptica\b",
    r"\bcrisis\s+convulsiva\b",
    r"\bconvulsión(es)?\b",
    r"\bconvulsivo\b",
]

# --- Compilar patrón ---
seizure_pat = re.compile("|".join(seizure_kw), flags=re.I)

# --- Crear variable binaria ---
df_nar['Seizure'] = df_nar['Narrative_clean'].apply(lambda t: 1 if isinstance(t, str) and seizure_pat.search(t) else 0)

# --- (Opcional) Verificación ---
# df['Seizure'].value_counts()
# df.loc[df['Seizure'] == 1, ['Report Id', 'Narrative and comments']].head()


In [64]:
columns_to_show = df_nar.columns.difference(['Narrative and comments', 'Narrative_clean'])
df_nar[columns_to_show]

,Admin_Route,Admin_Route_confidence,Admin_Route_source,Age,Gender,Infection_Type,Report Id,Seizure,Temp_C,Temp_Category,is_bacteria,is_fungus,is_parasite,is_virus,mentions_cephalosporin,route_im,route_inhaled,route_intraarticular,route_intraperitoneal,route_intrathecal,route_iv,route_ophthalmic,route_oral,route_otic,route_rectal,route_sc,route_topical,route_transdermal,route_vaginal
1,oral,medium,global,66.0,male,fungus,803589340,0,NaN,normal,0,1,0,0,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0
5,intrathecal,high,near-drug,14.0,male,non-specified,803562657,0,NaN,fever,0,0,0,0,True,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,multiple,high,near-drug,1.0,male,bacteria,803476613,0,NaN,fever,1,0,0,0,True,0,0,0,0,1,1,1,1,0,0,0,0,0,0
12,oral,high,near-drug,NaN,female,non-specified,803475361,0,NaN,non-specified,0,0,0,0,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0
14,intrathecal,high,near-drug,33.0,male,non-specified,803353779,0,40.0,fever,0,0,0,0,True,0,0,0,0,1,0,0,0,0,0,0,0,0,0
15,oral,high,near-drug,NaN,male,non-specified,803347905,0,NaN,non-specified,0,0,0,0,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0
18,oral,high,near-drug,NaN,female,fungus,803280356,0,NaN,non-specified,0,1,0,0,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0
23,oral,high,near-drug,86.0,female,bacteria,803165826,0,NaN,non-specified,1,0,0,0,True,0,0,0,0,0,0,0,1,0,0,0,0,0,0
24,non-specified,None,None,4.0,male,bacteria,803153340,0,NaN,non-specified,1,0,0,0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35,multiple,high,near-drug,51.0,male,non-specified,803058410,0,NaN,non-specified,0,0,0,0,True,0,0,0,0,0,1,0,1,0,0,0,0,0,0




Gravedad de reacción adversa (serious / non-serious)

Si puedes extraerlo, agrega una variable serious = 1/0 o una escala ordinal.


Temporalidad de la reacción

Si se puede obtener cuándo ocurrió la reacción después de iniciar el tratamiento (ej. en días), incluir time_to_event puede ser útil.



Número de comorbilidades

Una variable continua o categorizada (0, 1–2, ≥3) puede servir como control de complejidad clínica.

Visualización de matriz de correlaciones clínicas

Posteriormente, podrías visualizar correlaciones entre tipos de drogas, infecciones y comorbilidades versus convulsiones para ver asociaciones antes del modelado.